In [9]:
import pandas as pd
import os
import yaml

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from data_loader import *
from utils import *



In [2]:
# Config
config_path = 'config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)


In [ ]:
# Mysql string
db_url = config['MYSQL_STRING']

In [7]:
def write_df_to_db(df, table_name, db_type="postgres", db_url=None, if_exists="append", chunksize=5000):
    """
    Write a pandas DataFrame to either PostgreSQL or MySQL based on db_type.
    
    Args:
        df (pd.DataFrame): DataFrame to write
        table_name (str): Table name
        db_type (str): "postgres" or "mysql"
        db_url (str): Full SQLAlchemy DB URL
        if_exists (str): 'append', 'replace', or 'fail'
        chunksize (int): Number of rows per batch
    """
    import pandas as pd
    from sqlalchemy import create_engine
    from sqlalchemy.orm import sessionmaker

    if db_url is None:
        raise ValueError("db_url must be provided")

    # Convert unsupported types
    for col in df.columns:
        if pd.api.types.is_period_dtype(df[col]):
            df[col] = df[col].dt.to_timestamp()
        elif pd.api.types.is_timedelta64_dtype(df[col]):
            df[col] = df[col].dt.total_seconds()
        elif pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype(str)

    # Ensure the driver is correct
    if db_type.lower() == "postgres":
        # PostgreSQL usually uses psycopg2
        if "postgresql" not in db_url:
            db_url = "postgresql+psycopg2://" + db_url.split("://")[1]
    elif db_type.lower() == "mysql":
        # MySQL usually uses pymysql
        if "mysql" not in db_url:
            db_url = "mysql+pymysql://" + db_url.split("://")[1]
    else:
        raise ValueError("db_type must be 'postgres' or 'mysql'")

    # Create engine and session
    engine = create_engine(db_url)
    Session = sessionmaker(bind=engine)

    # Write using session manager
    with Session() as session:
        try:
            df.to_sql(
                table_name,
                con=session.connection(),
                if_exists=if_exists,
                index=False,
                chunksize=chunksize
            )
            session.commit()
        except Exception as e:
            session.rollback()
            print(f"Error writing to table '{table_name}': {e}")
            raise

In [4]:
# League dat
leagues_dat = get_leagues(home_dir +"/data/Leagues/leagues.parquet")
leagues_dat[['league_id','league_name','country_name']].sort_values('league_id')

,league_id,league_name,country_name
13,1,World Cup,World
17,2,UEFA Champions League,World
179,3,UEFA Europa League,World
0,4,Euro Championship,World
359,5,UEFA Nations League,World
...,...,...,...
1126,1129,ASEAN Club Championship,World
1127,1130,USL Super League,USA
1128,1131,Super Cup,Mongolia
1129,1132,AFC Challenge League,World


In [11]:
write_df_to_db(leagues_dat.iloc[:,1:], "league", db_type="mysql", db_url=db_url)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/2079050629.py:22: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df[col]):
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/2079050629.py:26: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[col]):


In [ ]:
# Add leagues to the database
leagues_dat.iloc[:,1:].to_sql('league',con = sql_engine,if_exists = 'replace',index = False)

In [12]:
teams_dat = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")
teams_dat.head()

,team_name,league
0,Angers,61
1,Lille,61
2,Lyon,61
3,Marseille,61
4,Nantes,61


In [13]:
# Add Team-legaue map to db
write_df_to_db(teams_dat, "team_league_map", db_type="mysql", db_url=db_url)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/2079050629.py:22: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df[col]):
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/2079050629.py:26: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[col]):


In [14]:
fixtures_dir = home_dir + "/data/Fixtures"

complete_data = pd.DataFrame()
for file in os.listdir(fixtures_dir):
    dat = pd.read_parquet(os.path.join(fixtures_dir,file))
    complete_data = pd.concat([complete_data,dat],axis = 0)

complete_data = complete_data.reset_index()
complete_data.drop(columns = ['index'],inplace=True)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  complete_data = pd.concat([complete_data,dat],axis = 0)
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/1208416677.py:6: FutureWarning: The behavior of DataFrame concatenatio

In [15]:
# Data checks
complete_data['passes_accuracy'] = complete_data['passes_accuracy'].astype("float64")
complete_data.rename(columns= {'passes_accuracy':'passes_accurate'},inplace =True)
complete_data['fixture_date'] = pd.to_datetime(complete_data.fixture_date)
complete_data['fixture_date_dt'] = complete_data['fixture_date'].dt.date
complete_data = create_datetime_columns(complete_data,'fixture_date')
complete_data['games_rating'] = pd.to_numeric(complete_data['games_rating'])

complete_data['season'] = complete_data['fixture_date'].apply(get_season)


# Targets
complete_data['outcome_num'] = pd.Categorical(complete_data.outcome).codes

complete_data['win'] = np.where(complete_data.outcome.str.lower() == 'win', 1,0)
complete_data['draw'] = np.where(complete_data.outcome.str.lower() == 'draw', 1,0)
complete_data['loss'] = np.where(complete_data.outcome.str.lower() == 'loss', 1,0)

# primary position map:
player_map = get_major_position(complete_data)

# Join back to complete_data
complete_data = pd.merge(complete_data,player_map,on = 'player_id',how = 'left')

# Joins:
complete_data = complete_data.merge(teams_dat.drop_duplicates(),how = 'left', left_on= 'team',right_on = 'team_name').drop(columns = ['team_name'])
complete_data = complete_data.merge(leagues_dat[['league_id','league_name']],how = 'left', left_on = 'league', right_on = 'league_id')


/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:50: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['month_e'] = data[dt_col].dt.to_period('M')
/Users/echhitjoshi/Library/Mobile Documents/com~apple~CloudDocs/Work/overperformXG/app/data_loader.py:54: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  data['week_e'] = data[dt_col].dt.to_period('W-MON')


In [16]:
complete_data.head()

,player_id,player_name,offsides,games_minutes,games_number,games_position,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_conceded,goals_assists,goals_saves,passes_total,passes_key,passes_accurate,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,fixture_id,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,opponent,fixture_date,team_winner,team,outcome,dribble_success_rate,target_shot_conversion_perc,duels_won_perc,pass_accuracy_perc,fixture_date_dt,year_e,month_e,month_name_e,day_of_week_e,day_of_week_name_e,week_e,season,outcome_num,win,draw,loss,major_position,league,league_id,league_name
0,963,Lukáš Hrádecký,NaN,90.0,1,G,8.9,True,False,NaN,NaN,NaN,0,NaN,6.0,39.0,NaN,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,1.0,719361,4,4.0,2,0,0.0,0,Borussia Monchengladbach,2021-08-21 16:30:00+00:00,Bayer Leverkusen,Bayer Leverkusen,win,NaN,NaN,NaN,74.358974,2021-08-21,2021.0,2021-08,August,5.0,Saturday,2021-08-17,2021/2022,2,1,0,0,G,78,78,Bundesliga
1,152654,Jeremie Frimpong,NaN,90.0,30,D,7.0,False,False,NaN,NaN,NaN,0,NaN,NaN,32.0,1.0,27.0,2.0,NaN,NaN,10.0,5.0,2.0,1.0,1.0,2.0,1.0,0,0,NaN,NaN,0,0,NaN,719361,4,4.0,2,0,0.0,0,Borussia Monchengladbach,2021-08-21 16:30:00+00:00,Bayer Leverkusen,Bayer Leverkusen,win,50.0,NaN,50.000000,84.375000,2021-08-21,2021.0,2021-08,August,5.0,Saturday,2021-08-17,2021/2022,2,1,0,0,M,78,78,Bundesliga
2,48119,Odilon Kossounou,NaN,90.0,6,D,7.6,False,False,NaN,NaN,NaN,0,NaN,NaN,57.0,1.0,46.0,2.0,1.0,2.0,6.0,5.0,NaN,NaN,1.0,1.0,NaN,0,0,NaN,NaN,0,0,NaN,719361,4,4.0,2,0,0.0,0,Borussia Monchengladbach,2021-08-21 16:30:00+00:00,Bayer Leverkusen,Bayer Leverkusen,win,NaN,NaN,83.333333,80.701754,2021-08-21,2021.0,2021-08,August,5.0,Saturday,2021-08-17,2021/2022,2,1,0,0,D,78,78,Bundesliga
3,972,Jonathan Tah,NaN,90.0,4,D,7.2,False,False,NaN,NaN,NaN,0,1.0,NaN,54.0,1.0,51.0,2.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,2.0,0,0,NaN,NaN,0,0,NaN,719361,4,4.0,2,0,0.0,0,Borussia Monchengladbach,2021-08-21 16:30:00+00:00,Bayer Leverkusen,Bayer Leverkusen,win,NaN,NaN,60.000000,94.444444,2021-08-21,2021.0,2021-08,August,5.0,Saturday,2021-08-17,2021/2022,2,1,0,0,D,78,78,Bundesliga
4,530,Mitchel Bakker,NaN,90.0,5,D,8.2,False,False,1.0,1.0,1.0,0,1.0,NaN,46.0,1.0,40.0,1.0,NaN,1.0,10.0,6.0,4.0,3.0,NaN,2.0,2.0,1,0,NaN,1.0,0,0,NaN,719361,4,4.0,2,0,0.0,0,Borussia Monchengladbach,2021-08-21 16:30:00+00:00,Bayer Leverkusen,Bayer Leverkusen,win,75.0,100.0,60.000000,86.956522,2021-08-21,2021.0,2021-08,August,5.0,Saturday,2021-08-17,2021/2022,2,1,0,0,D,78,78,Bundesliga


In [18]:
write_df_to_db(complete_data,"complete_data",db_type="mysql", db_url=db_url)

/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/2079050629.py:22: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if pd.api.types.is_period_dtype(df[col]):
/var/folders/x2/4jp5z9p56hd1r0cvzk8fb0200000gn/T/ipykernel_7166/2079050629.py:26: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(df[col]):


In [19]:
fixture_dat = calculate_fixture_stats(complete_data,['league_name'])

In [20]:
fixture_dat.head()

,fixture_id,team,league_name,offsides,games_minutes,games_number,games_rating,games_captain,games_substitute,shots_total,shots_on,goals_total,goals_assists,goals_saves,passes_total,passes_key,passes_accurate,tackles_total,tackles_blocks,tackles_interceptions,duels_total,duels_won,dribbles_attempts,dribbles_success,dribbles_past,fouls_drawn,fouls_committed,cards_yellow,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved,dribble_success_rate,target_shot_conversion_perc,duels_won_perc,pass_accuracy_perc,win,draw,loss,team_goals_scored,team_non_penalty_goals_scored,team_goals_scored_half,team_goals_conceded,team_non_penalty_goals_conceded,team_goals_conceded_half,team_winner,opponent,goal_diff,week_e,year_e,season,fixture_date,l3_win,l3_draw,l3_loss,l5_win,l5_draw,l5_loss
22288,1223983,1. FC Heidenheim,Bundesliga,0.0,60.000000,16.277778,6.900000,0.055556,0.444444,5.0,3.0,2.0,2.0,0.0,312.0,4.0,222.0,17.0,1.0,8.0,118.0,58.0,19.0,8.0,4.0,9.0,15.0,3,0,0.0,0.0,0,0,0.0,73.333333,100.000000,55.874642,69.948108,1,0,0,2,2.0,0,0,0.0,0,1. FC Heidenheim,FC St. Pauli,2,2024-08-20,2024.0,2024/2025,2024-08-25 15:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
22298,1223988,1. FC Heidenheim,Bundesliga,1.0,60.000000,16.277778,7.353333,0.055556,0.444444,13.0,8.0,4.0,2.0,0.0,268.0,10.0,207.0,19.0,4.0,12.0,108.0,56.0,21.0,10.0,5.0,9.0,15.0,1,0,0.0,0.0,1,0,0.0,76.666667,83.333333,52.482401,78.213158,1,0,0,4,4.0,2,0,0.0,0,1. FC Heidenheim,FC Augsburg,4,2024-08-27,2024.0,2024/2025,2024-09-01 13:30:00+00:00,1.0,0.0,0.0,1.0,0.0,0.0
22310,1223994,1. FC Heidenheim,Bundesliga,1.0,60.466667,16.777778,6.580000,0.055556,0.444444,6.0,4.0,2.0,1.0,0.0,309.0,8.0,242.0,17.0,4.0,5.0,98.0,44.0,10.0,5.0,11.0,11.0,18.0,2,0,1.0,1.0,1,0,0.0,80.000000,75.000000,55.983738,74.652862,0,0,1,2,2.0,1,4,4.0,3,Borussia Dortmund,Borussia Dortmund,-2,2024-09-10,2024.0,2024/2025,2024-09-13 18:30:00+00:00,2.0,0.0,0.0,2.0,0.0,0.0
22332,1224005,1. FC Heidenheim,Bundesliga,1.0,60.000000,16.666667,6.553333,0.055556,0.444444,6.0,1.0,0.0,0.0,0.0,337.0,6.0,254.0,7.0,4.0,11.0,66.0,31.0,9.0,2.0,6.0,7.0,6.0,1,0,0.0,0.0,0,0,0.0,66.666667,0.000000,49.606481,76.309731,0,0,1,0,0.0,0,3,3.0,0,SC Freiburg,SC Freiburg,-3,2024-09-17,2024.0,2024/2025,2024-09-21 13:30:00+00:00,2.0,0.0,1.0,2.0,0.0,1.0
22356,1224017,1. FC Heidenheim,Bundesliga,2.0,64.071429,17.555556,7.207143,0.055556,0.444444,12.0,6.0,2.0,2.0,0.0,359.0,14.0,270.0,20.0,3.0,8.0,126.0,69.0,12.0,7.0,4.0,10.0,10.0,4,1,0.0,0.0,0,0,0.0,76.666667,50.000000,53.020851,70.394965,1,0,0,2,2.0,1,0,0.0,0,1. FC Heidenheim,FSV Mainz 05,2,2024-09-24,2024.0,2024/2025,2024-09-28 13:30:00+00:00,1.0,0.0,2.0,2.0,0.0,2.0


In [ ]:
complete_data = pd.read_sql("select * from public.complete_data", con = config['DB_STRING'])

In [ ]:
complete_data.head()